In [1]:
import requests 
import re 
import pickle

with open(파일명, "r", encoding='utf-8') as f:
    for line in f:
        print(line)

In [2]:
with open("./kbo.pkl", "rb" ) as f:
    kbo= pickle.load(f)

In [134]:
from bs4 import BeautifulSoup
#url = "https://www.koreabaseball.com/Futures/Player/HitterDetail.aspx?playerId={}"
url = "https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId={}"
r = requests.get(url.format('51897'))
bs = BeautifulSoup(r.text, 'html.parser')
tmp_dict = dict()
for x in bs.find("div", {'class' :'player_basic'}).findAll("li"):
    key, value = x.text.split(":")
    tmp_dict[key.strip()] = value.strip()
tmp_dict['team'] = bs.find("h4", id="h4Team").text

In [135]:
r.url

'https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId=51897'

In [136]:
tmp_dict

{'선수명': '조병현',
 '등번호': 'No.41',
 '생년월일': '2002년 05월 08일',
 '포지션': '투수(우투우타)',
 '신장/체중': '182cm/90kg',
 '경력': '온양온천초-온양중-세광고-SK-SSG-상무',
 '입단 계약금': '7000만원',
 '연봉': '3000만원',
 '지명순위': '21 SK 2차 3라운드 28순위',
 '입단년도': '21SK',
 'team': '상무 '}

In [137]:
import tqdm
total = []
for player_id in tqdm.tqdm(kbo):
    r = requests.get(url.format(player_id))
    bs = BeautifulSoup(r.text, 'lxml')
    tmp_dict = dict()
    for x in bs.find("div", {'class' :'player_basic'}).findAll("li"):
        key, value = x.text.split(":")
        tmp_dict[key.strip()] = value.strip()
    tmp_dict['team'] = bs.find("h4", id="h4Team").text
    total.append(tmp_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 882/882 [01:59<00:00,  7.40it/s]


In [6]:
total

In [31]:
pd.DataFrame(total).to_excel("./kbo.xlsx")

In [138]:
for x in total:
    print(x)
    break

{'선수명': '강매성', '등번호': 'No.95', '생년월일': '2003년 06월 08일', '포지션': '투수(우투우타)', '신장/체중': '187cm/94kg', '경력': '상탄초(일산동구리틀)-개군중-공주고', '입단 계약금': '5000만원', '연봉': '3000만원', '지명순위': '22 SSG 2차 6라운드 52순위', '입단년도': '', 'team': 'SSG 랜더스'}


In [139]:
height, weight = x['신장/체중'].split("/")

In [140]:
height

'187cm'

In [141]:
weight

'94kg'

In [142]:
def calc(h, w):
    return int(w.replace("kg", "")) / (int(h.replace('cm','')) / 100) ** 2

In [143]:
calc(height, weight)

26.88095170007721

In [144]:
(lambda h , w:int(w.replace("kg", "")) / (int(h.replace('cm','')) / 100) ** 2)(height,weight)

26.88095170007721

In [145]:
for x in total:
    height, weight = x['신장/체중'].split("/")
    #x['BMI'] = (lambda h , w:int(w.replace("kg", "")) / (int(h.replace('cm','')) / 100) ** 2)(height,weight)
    x['BMI'] = calc(height,weight)

In [146]:
sorted({x['선수명'] : x['BMI'] for x in total}.items(), key=lambda x : x[1])

[('신제왕', 19.623233908948194),
 ('강한울', 20.1459051921492),
 ('백선기', 20.23355301190889),
 ('김성표', 20.286507911738084),
 ('이태규', 20.37120869171571),
 ('엄상백', 20.58966513197403),
 ('정주후', 20.679012345679013),
 ('김태윤', 20.761245674740486),
 ('구자욱', 20.99605274208449),
 ('윤강찬', 21.0372534696859),
 ('오태양', 21.146319909102385),
 ('정구범', 21.20099137030069),
 ('고명성', 21.461936624163616),
 ('채지선', 21.604938271604937),
 ('강상원', 21.63331530557058),
 ('이상혁', 21.63331530557058),
 ('박소준', 21.705129432793896),
 ('윤산흠', 21.705129432793896),
 ('지명성', 21.71806608974573),
 ('최수호', 21.798202394816204),
 ('최현준', 21.798202394816204),
 ('김규성', 21.798202394816204),
 ('신범준', 21.83589485176787),
 ('임도혁', 21.85727788279773),
 ('권민석', 21.85727788279773),
 ('오정환', 21.913805697589478),
 ('박신지', 21.913805697589478),
 ('안재석', 21.913805697589478),
 ('오재원', 21.913805697589478),
 ('박상언', 21.913805697589478),
 ('안주형', 21.952479338842977),
 ('조현진', 22.03840115928028),
 ('우규민', 22.152646502835537),
 ('김상수', 22.20408163265306

In [128]:
x

{'선수명': '조병현',
 '등번호': 'No.41',
 '생년월일': '2002년 05월 08일',
 '포지션': '투수(우투우타)',
 '신장/체중': '182cm/90kg',
 '경력': '온양온천초-온양중-세광고-SK-SSG-상무',
 '입단 계약금': '7000만원',
 '지명순위': '21 SK 2차 3라운드 28순위',
 '입단년도': '21SK',
 'team': '상무 ',
 'BMI': 27.17063156623596}

In [165]:
cnt = 0 
for x in total:
    try:
        if x['연봉'].find("달러") > -1:
            print(x)
    except:
        cnt += 1


{'선수명': '노바', '등번호': 'No.', '생년월일': '1987년 01월 12일', '포지션': '투수(우투우타)', '신장/체중': '193cm/102kg', '경력': '도미니카 Lic.Vespertino Lucila Mojica(고)', '입단 계약금': '150000달러', '연봉': '750000달러', '지명순위': '22 SSG 자유선발', '입단년도': '22SSG', 'team': 'SSG 랜더스', 'BMI': 27.383285457327716}
{'선수명': '라가레스', '등번호': 'No.27', '생년월일': '1989년 03월 17일', '포지션': '외야수(우투우타)', '신장/체중': '187cm/99kg', '경력': '도미니카 Juan Pablo(고)', '입단 계약금': '', '연봉': '400000달러', '지명순위': '22 SSG 자유선발', '입단년도': '22SSG', 'team': 'SSG 랜더스', 'BMI': 28.310789556464293}
{'선수명': '모리만도', '등번호': 'No.33', '생년월일': '1992년 11월 20일', '포지션': '투수(좌투좌타)', '신장/체중': '182cm/93kg', '경력': '미국 Ocean Lakes(고)', '입단 계약금': '', '연봉': '180000달러', '지명순위': '22 SSG 자유선발', '입단년도': '22SSG', 'team': 'SSG 랜더스', 'BMI': 28.076319285110493}
{'선수명': '크론', '등번호': 'No.', '생년월일': '1993년 02월 17일', '포지션': '내야수(우투우타)', '신장/체중': '196cm/115kg', '경력': '미국 Mountain Pointe(고)-미국 Texas Christian(대)', '입단 계약금': '150000달러', '연봉': '600000달러', '지명순위': '22 SSG 자유선발', '입단년도': '22SSG', 'team': 'SSG

In [148]:
cnt

0

In [57]:
cnt

42

In [58]:
hana = "https://www.kebhana.com/cms/rate/wpfxd651_01i_01.do"
hana_payload = {"ajax"  : "true",
"curCd"  : "",
"tmpInqStrDt"  : "2023-01-12",
"pbldDvCd"  : "0",
"pbldSqn"  : "",
"hid_key_data"  : "",
"inqStrDt"  : "20230112",
"inqKindCd"  : "1",
"hid_enc_data"  : "",
"requestTarget"  : "searchContentDiv",}
r = requests.post(hana, data=hana_payload)

In [90]:
for idx, x in enumerate(BeautifulSoup(r.text, 'lxml').findAll("tr")[3].findAll("td")):
    print(idx, x.text)

In [93]:
BeautifulSoup(r.text, 'lxml').findAll("tr")[3].findAll("td")[9].text

'1,243.00'

In [ ]:
def exchange(date_):
    return 해당날짜의 매매기준율 환율 

In [95]:
def tmp(date_):
    """
        date_ 형식은 yyyy-mm-dd 형식으로 해주세요 
    """
    pass

In [96]:
help(tmp)

Help on function tmp in module __main__:

tmp(date_)
    date_ 형식은 yyyy-mm-dd 형식으로 해주세요



In [97]:
hana = "https://www.kebhana.com/cms/rate/wpfxd651_01i_01.do"
hana_payload = {"ajax"  : "true",
"curCd"  : "",
"tmpInqStrDt"  : "2023-01-12",
"pbldDvCd"  : "0",
"pbldSqn"  : "",
"hid_key_data"  : "",
"inqStrDt"  : "20230112",
"inqKindCd"  : "1",
"hid_enc_data"  : "",
"requestTarget"  : "searchContentDiv",}

In [98]:
def exchange(date_):
    """
        date_ 형식은 yyyy-mm-dd 형식으로 해주세요 
    """
    hana_payload['tmpInqStrDt'] = date_ 
    hana_payload['inqStrDt'] = date_.replace("-", "")
    r = requests.post(hana, data = hana_payload)
    return BeautifulSoup(r.text, 'lxml').findAll("tr")[3].findAll("td")[9].text

In [100]:
from datetime import date, datetime, timedelta
date_dict = dict()
for x in range(3):
    key = (date(2022,1,1) + timedelta(days=x)).strftime("%Y-%m-%d")
    val = exchange((date(2022,1,1) + timedelta(days=x)).strftime("%Y-%m-%d"))
    date_dict[key] = val

In [126]:
date_dict

{'2022-01-01': '1,190.50', '2022-01-02': '1,190.50', '2022-01-03': '1,193.50'}

In [118]:
exchange((date(2022,1,1) + timedelta(days=x)).strftime("%Y-%m-%d"))

'1,190.00'

In [152]:
with open("./kbo_result.pkl", "rb") as f:
    kbo_result = pickle.load(f)

In [166]:
rt = exchange('2023-01-01')
exchange_rate = float(rt.replace(",", ""))

In [172]:
for x in kbo_result:
    try:
        if x['연봉'].find("달러") > -1:
            x['연봉(만원)'] = int(x['연봉'].replace("달러", "")) * exchange_rate
        else:
            x['연봉(만원)'] = int(x['연봉'].replace("만원", "")) * 10000
    except:
        x['연봉(만원)'] = 0

In [178]:
for x in kbo_result:
    print("{} - {:,}".format(x['선수명'], x['연봉(만원)'] ))

강매성 - 30,000,000
고명준 - 30,000,000
고효준 - 40,000,000
김강민 - 160,000,000
김건우 - 31,000,000
김광현 - 8,100,000,000
김교람 - 30,000,000
김규남 - 30,000,000
김도현 - 30,000,000
김민식 - 95,000,000
김민재 - 31,000,000
김상수 - 300,000,000
김성민 - 30,000,000
김성현 - 200,000,000
김재현 - 40,000,000
김정우 - 30,000,000
김주온 - 30,000,000
김태우 - 30,000,000
김태윤 - 30,000,000
김태훈 - 150,000,000
김택형 - 125,000,000
노경은 - 100,000,000
노바 - 947,250,000.0
라가레스 - 505,200,000.0
모리만도 - 227,340,000.0
문승원 - 1,600,000,000
박민호 - 120,000,000
박상후 - 30,000,000
박성한 - 140,000,000
박시후 - 30,000,000
박정빈 - 30,000,000
박제범 - 30,000,000
박종훈 - 1,800,000,000
백승건 - 30,000,000
서동민 - 40,000,000
서상준 - 30,000,000
서진용 - 185,000,000
석정우 - 30,000,000
신동민 - 30,000,000
신재영 - 40,000,000
신헌민 - 30,000,000
안상현 - 32,000,000
오원석 - 65,000,000
오준혁 - 32,000,000
오태곤 - 105,000,000
유서준 - 30,000,000
유호식 - 30,000,000
윤태현 - 30,000,000
이거연 - 30,000,000
이건욱 - 32,000,000
이기순 - 30,000,000
이재원 - 1,000,000,000
이정범 - 32,000,000
이태양 - 120,000,000
이현석 - 50,000,000
이흥련 - 90,000,000
이희재 - 30,000,00

In [184]:
sorted({x['선수명'] : x['연봉(만원)'] for x in kbo_result}.items(), key=lambda x : x[1], reverse=True)

[('김광현', 8100000000),
 ('추신수', 2700000000),
 ('구자욱', 2500000000),
 ('한유섬', 2400000000),
 ('루친스키', 2020800000.0),
 ('미란다', 2020800000.0),
 ('나성범', 2000000000),
 ('박건우', 1900000000),
 ('박종훈', 1800000000),
 ('문승원', 1600000000),
 ('오승환', 1600000000),
 ('손아섭', 1500000000),
 ('김재환', 1500000000),
 ('폰트', 1389300000.0),
 ('뷰캐넌', 1389300000.0),
 ('최정', 1200000000),
 ('허경민', 1200000000),
 ('요키시', 1136700000.0),
 ('켈리', 1136700000.0),
 ('쿠에바스', 1010400000.0),
 ('양현종', 1000000000),
 ('양의지', 1000000000),
 ('안치홍', 1000000000),
 ('최재훈', 1000000000),
 ('노바', 947250000.0),
 ('최형우', 900000000),
 ('푸이그', 884100000.0),
 ('데스파이네', 884100000.0),
 ('수아레즈', 884100000.0),
 ('킹험', 884100000.0),
 ('터크먼', 884100000.0),
 ('라모스', 820950000.0),
 ('백정현', 800000000),
 ('이대호', 800000000),
 ('정우람', 800000000),
 ('크론', 757800000.0),
 ('루이즈', 757800000.0),
 ('피렐라', 757800000.0),
 ('피터스', 757800000.0),
 ('페르난데스', 757800000.0),
 ('이정후', 750000000),
 ('박석민', 700000000),
 ('최주환', 650000000),
 ('플럿코', 631500000.0),
 ('소크라테스', 

In [193]:
print("{:,}원".format(sum([x['연봉(만원)'] for x in kbo_result]) / len(kbo_result)))

145,283,273.3469388원


In [195]:
for x in kbo_result:
     x['나이'] = 2023 - int(x['생년월일'].split("년")[0])

In [198]:
sorted({x['선수명'] : x['나이'] for x in kbo_result}.items(), key=lambda x : x[1])

[('주승빈', 19),
 ('정정우', 19),
 ('김서진', 19),
 ('김시완', 19),
 ('강매성', 20),
 ('김태윤', 20),
 ('박상후', 20),
 ('윤태현', 20),
 ('이기순', 20),
 ('최유빈', 20),
 ('김리안', 20),
 ('노운현', 20),
 ('박정훈', 20),
 ('박찬혁', 20),
 ('백진수', 20),
 ('송정인', 20),
 ('윤석원', 20),
 ('김성우', 20),
 ('김유민', 20),
 ('김주완', 20),
 ('문정빈', 20),
 ('양진혁', 20),
 ('엄태경', 20),
 ('이주헌', 20),
 ('이지훈', 20),
 ('임준형', 20),
 ('조원태', 20),
 ('최원영', 20),
 ('권성준', 20),
 ('김병준', 20),
 ('박영현', 20),
 ('안현민', 20),
 ('우종휘', 20),
 ('이상우', 20),
 ('최동희', 20),
 ('한지웅', 20),
 ('강병우', 20),
 ('김도영', 20),
 ('김찬민', 20),
 ('윤도현', 20),
 ('이성주', 20),
 ('최지민', 20),
 ('한승연', 20),
 ('김녹원', 20),
 ('박준영', 20),
 ('서준교', 20),
 ('이준혁', 20),
 ('이한', 20),
 ('이현우', 20),
 ('임지민', 20),
 ('강도훈', 20),
 ('김상민', 20),
 ('김서준', 20),
 ('김세민', 20),
 ('김영웅', 20),
 ('신정환', 20),
 ('윤정훈', 20),
 ('이병헌', 20),
 ('이재현', 20),
 ('조민성', 20),
 ('김용완', 20),
 ('엄장윤', 20),
 ('윤동희', 20),
 ('조세진', 20),
 ('진승현', 20),
 ('하혜성', 20),
 ('한태양', 20),
 ('강산', 20),
 ('신민철', 20),
 ('윤태호', 20),
 ('이원재', 20),
 ('전희범',

In [199]:
stat_dict = dict()
for x in kbo_result:
    if x['나이'] in stat_dict:
        stat_dict[x['나이']] += 1
    else:
        stat_dict[x['나이']] = 1

In [202]:
sorted(stat_dict.items(), key=lambda x : x[1], reverse=True)

[(24, 93),
 (20, 78),
 (21, 66),
 (23, 66),
 (22, 62),
 (25, 62),
 (27, 53),
 (26, 53),
 (28, 51),
 (29, 47),
 (34, 41),
 (31, 40),
 (33, 39),
 (30, 35),
 (32, 26),
 (35, 19),
 (36, 15),
 (38, 11),
 (37, 7),
 (39, 6),
 (41, 4),
 (19, 4),
 (40, 3),
 (44, 1)]

In [34]:
temp_dic = {"called": "0"}

def fib(n):
    temp_dic["called"] = str(int(temp_dic["called"]) +1)
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [37]:
temp_dic

{'called': '287'}

In [36]:
fib(11)

144

In [209]:
a = [1,2,3]

In [229]:
class Stack:
    
    def __init__(self):
        self.items = []
        
    def push(self, value):
        self.items.append(value)
        
    def pop(self):
        value = self.items.pop()
        if value is not None:
            return value
        else:
            print("stack is empty")
    def __repr__(self):
        return repr(self.items)

In [230]:
stack1 = Stack()
stack2 = Stack()

In [231]:
stack1.push(10)

In [232]:
print(stack1)

[10]


In [234]:
print(stack2)

[]


In [235]:
stack2.push(20)

In [236]:
stack2.push(50)

In [237]:
print(stack1)

[10]


In [261]:
import pandas as pd
class crawling:
    def __init__(self):
        self.header = {'user-agent': 
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                       }
        
    def get_naver_stock(self, code):
        url = f"https://finance.naver.com/item/sise_day.naver?code={code}&page=1"
        r = requests.get(url, headers=self.header)
        return pd.read_html(r.text)[0].dropna()
        

In [262]:
crawler = crawling()

In [263]:
crawler.get_naver_stock('005930')

,날짜,종가,전일비,시가,고가,저가,거래량
1,2023.01.30,63300.0,1300.0,64900.0,64900.0,63100.0,19146472.0
2,2023.01.27,64600.0,700.0,64400.0,65000.0,63900.0,18760182.0
3,2023.01.26,63900.0,500.0,63800.0,63900.0,63300.0,13278277.0
4,2023.01.25,63400.0,1600.0,63500.0,63700.0,63000.0,16822710.0
5,2023.01.20,61800.0,300.0,62100.0,62300.0,61100.0,9646327.0
9,2023.01.19,61500.0,1100.0,60500.0,61500.0,60400.0,12808490.0
10,2023.01.18,60400.0,600.0,60700.0,61000.0,59900.0,11584041.0
11,2023.01.17,61000.0,100.0,61200.0,61500.0,60600.0,9831456.0
12,2023.01.16,61100.0,300.0,61300.0,61600.0,60800.0,10039972.0
13,2023.01.13,60800.0,300.0,60500.0,61200.0,60400.0,12510328.0


In [264]:
a = list()

In [248]:
dir(a)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']